This is an implementation of the Central Limit Theorem and Bootstrap for the Linkage Model. Parts of this have been written by Ferdinand Suchanek.

Tasks:
- Compare the Bootstrap results to the Central Limit Theorem
- Therefore: Think about a nice metric/nice way to depict the results

In [1]:
from scipy.stats import dirichlet, binom, norm
import numpy as np
import random
# Simulation of the Model
# Simulate the allele frequencies
def create_p(M, K):
    temp = np.random.uniform(0, 1, size=(K, M))
    return temp

# Simulate the Individuals
def create_sample_pbekannt(M, N, K, p, q):
    N, K = q.shape
    K, M = p.shape 
    x = np.zeros((N, M))
    loc = np.dot(q, p)
    for m in range(M):
        for n in range(N):
            x[n, m] = binom.rvs(n=2, p=loc[n, m])
    return x  


# MLE (method from Pfaffelhuber instead of scipy.minimize)
def get_admixture_proportions(x, p, tol=1e-6):
    K, M = p.shape
    res = dirichlet.rvs(alpha=np.ones(K))
    err = 1
    while err > tol:
        loc = fun2(res, p, x)
        err = np.sum(np.abs(res - loc))
        res = loc
    return res

def fun2(q, p, loc_x):
    K, M = p.shape
    E = np.zeros((K, M))
    loc = np.dot(q, p)
    loc[loc==0] = 1e-16
    loc[loc==1] = 1-1e-16
    for k in range(K):
        E[k, :] = (loc_x * p[k, :] / loc + (2 - loc_x) * (1 - p[k, :]) / (1 - loc))
    res = np.sum(E, axis=1) / M * q / 2
    return res / np.sum(res)

# Example Application
# Simulate the true IAs
N = 100 # Number of individuals
K = 2 # Number of Populations
# True parameter
q_0 = np.zeros(2)
q_0[0] = 0.7
q_0[1] = 0.3
q = np.zeros((N, K))
M = 200 # Number of markers
p = create_p(M, K)
# Simulate the IAs
for n in range(N):
    q[n, :] = q_0

# Rows: Individuals
# Columns: Marker
x = create_sample_pbekannt(M, N, K, p, q)

# Find MLE
hat_q = get_admixture_proportions(x[0], p, tol=1e-6)
# Compare truth to estimation
print("Estimation and Truth",hat_q, q[1])

Estimation and Truth [[0.62252916 0.37747084]] [0.7 0.3]


# Central Limit Theorem

In [6]:
# Implementation of the Central Limit Theorem
def res(q0, p1):
    K = len(q0)
    if(min(q0) > 0):
        theta1 = np.dot(q0, p1)
        p1 = np.array(p1)
        p = np.array(p1 - p1[-1])
        p2 = np.outer(p[:-1],p[:-1])
        if(theta1 > 0 and theta1 < 1):
            res4 = 2*p2*(1/theta1) + 2*p2/(1-theta1)
        else:
            res4 = 2*p2
    else:
        res4 = np.zeros((K, K))
    return res4

def res_alle(M, q, p):
    res_int = 0
    for i in range(M):
        res_int += res(q, p[i])
    return np.linalg.inv(res_int/M)

# Calculates the asymptotic covariance
def cov(p_K5, q_K5, K, M):
    M = len(p_K5)
    N = len(q_K5)
    v = res_alle(M, q_K5[0], p_K5)
    for i in range(1,N):
        v += res_alle(M, q_K5[i], p_K5)
    cov_X5_X = -np.sum(v, axis=0)
    var_X5 = np.sum(v)
    C_extended = np.zeros((K, K))
    C_extended[:(K-1), :(K-1)] = v    
    C_extended[(K-1), :(K-1)] = cov_X5_X
    C_extended[:(K-1), (K-1)] = cov_X5_X    
    C_extended[(K-1), (K-1)] = var_X5  
    
    return C_extended/M

In [4]:
alpha = 0.01
hat_q = get_admixture_proportions(x[0], p, tol=1e-6)
covmatrix_estimation_clt = cov(p.T,hat_q.tolist(),2, M)
variance_clt = covmatrix_estimation_clt[0,0]

# Bootstrap

In [5]:
# Numver of Bootstrap Examples
n_bootstraps = 10
# Index of the individual
i = 0

hat_q = get_admixture_proportions(x[i], p, tol=1e-6)

q_estimation = []
for j in range(0,n_bootstraps):
    sample_with_replacement = [random.randint(0, M-1) for _ in range(M)]
    x_resampled = x[i][sample_with_replacement]
    p_resampled = p[:,sample_with_replacement]
    hat_q_resampled = get_admixture_proportions(x_resampled, p_resampled, tol=1e-6)
    q_estimation.append(hat_q_resampled[0][0])

